In [ ]:
import numpy as np
import xarray as xr
from scipy.interpolate import RegularGridInterpolator
import functions as md
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [ ]:
# --- Parâmetros do MDE / integração (iguais ao .m) ---
res_deg = 0.008333333333333333868   # ~30 arcsec
raio_integracao_m = 100_000         # 100 km

# --- Arquivos (ajuste os caminhos se você moveu para data/raw) ---
path_dem   = "../data/raw/MDS_MERIT_SRTM15PLUS_900m_fill.tif"
path_pts   = "../data/raw/Pontos_calculo_grade.txt"

# 1) Ler MDE como DataArray (y=lat, x=lon, graus)
da_H = md.geotiff_to_dataarray(path_dem)  # usa sua função; unidades em metros
H = da_H.values.astype(float)

lat_dem = np.asarray(da_H.y.values, dtype=float)  # S→N (crescente)
lon_dem = np.asarray(da_H.x.values, dtype=float)  # W→E (crescente)
lin, col = H.shape

print("DEM shape:", H.shape, "| lat range:", lat_dem.min(), "→", lat_dem.max(),
      "| lon range:", lon_dem.min(), "→", lon_dem.max())

# 2) Ler pontos de cálculo (lat, lon). Aceita comentários iniciados por '%'
Pontos = np.loadtxt(path_pts, comments='%', dtype=float)
latP_deg = Pontos[:, 0]
lonP_deg = Pontos[:, 1]

# 3) Interpolar H nos pontos P (bilinear estável)
interp_H = RegularGridInterpolator(
    (lat_dem, lon_dem), H, method="linear", bounds_error=False, fill_value=np.nan
)
HP = interp_H(np.column_stack([latP_deg, lonP_deg]))  # metros

print(f"HP stats (m): min={np.nanmin(HP):.2f}, max={np.nanmax(HP):.2f}")

In [ ]:
# Gravidade normal no elipsoide nos P (m/s²)
phiP_rad = np.deg2rad(latP_deg)
sinP = np.sin(phiP_rad)
gama0 = 9.7803267715 * (
    1
    + 0.0052790414 * (sinP**2)
    + 0.0000232718 * (sinP**4)
    + 0.0000001262 * (sinP**6)
    + 0.0000000007 * (sinP**8)
)

# Constantes físicas e geométricas
G   = 6.67259e-11      # m^3 kg^-1 s^-2 (usada no .m)
rho = 2670.0           # kg m^-3
R   = 6_371_000.0      # m

# Conversões "à la MATLAB"
metros = 3600 * 30     # 108000 m por grau (aprox. fixa)
res_metros = res_deg * 108000.0
denominador = 3 * (R**2)

# Pré-alocação
nP = latP_deg.size
EI = np.zeros(nP, dtype=float)

# Coeficientes do .m:
# cte0 depende de HP (vetor), cte1 e cte2 são escalares
cte0 = -np.pi * G * rho * (HP**2)            # m^2/s^2
cte1 = -(G * rho / 6.0) * (res_metros**2)    # m^2/s^2 * m^(-3) * m^2 = m^2/s^2 por termos normalizados
cte2 = (3.0 * G * rho / 40.0) * (res_metros**2)

print(f"cte1={cte1:.3e}, cte2={cte2:.3e}")


In [ ]:
# Construir grelhas de coordenadas do DEM (graus), iguais às do MATLAB
gradeMDA_lat = np.tile(lat_dem[:, None], (1, col))  # (lin, col)
gradeMDA_long = np.tile(lon_dem[None, :], (lin, 1)) # (lin, col)

# Loop nos pontos P
for ii in range(nP):
    # Distância "plana" aproximada (m) a partir de graus
    dlat = (gradeMDA_lat - latP_deg[ii]) * metros
    dlon = (gradeMDA_long - lonP_deg[ii]) * metros
    lo = np.sqrt(dlat*dlat + dlon*dlon)  # (lin, col) em metros

    # Máscara pelo raio de integração
    mask = lo <= raio_integracao_m
    if not np.any(mask):
        EI[ii] = 0.0
        continue

    # Vetorização: achatar e manter só dentro do raio
    lo_sel = lo[mask].astype(float)

    # Correção de curvatura: l = l + l^3 / (3 R^2)
    lo_corr = lo_sel + (lo_sel**3) / denominador

    H_sel = H[mask].astype(float)

    # Termos do integrando (iguais ao .m)
    # cte1 * sum( (H^3 - HP^3) / l^3 ) + cte2 * sum( (H^5 - HP^5) / l^5 )
    H3_diff = (H_sel**3) - (HP[ii]**3)
    H5_diff = (H_sel**5) - (HP[ii]**5)

    EI[ii] = cte1 * np.sum(H3_diff / (lo_corr**3)) + cte2 * np.sum(H5_diff / (lo_corr**5))

# Conversões finais
EI_N  = EI / gama0                     # m  (efeito indireto em N)
EI_Ag = (EI + cte0) * (0.3086) / gama0 # mGal (efeito indireto em Δg)

print(f"EI_N (m): min={np.nanmin(EI_N):.3e}, max={np.nanmax(EI_N):.3e}")
print(f"EI_Ag (mGal): min={np.nanmin(EI_Ag):.3e}, max={np.nanmax(EI_Ag):.3e}")


In [ ]:

out_csv = "../data/processed/EI_pontos.csv"
df_EI = pd.DataFrame({
    "lat": latP_deg,
    "lon": lonP_deg,
    "EI_N_m": EI_N,
    "EI_Ag_mGal": EI_Ag,
})
df_EI.to_csv(out_csv, index=False)
print(f"[OK] salvo: {out_csv}")


In [ ]:
plt.figure(figsize=(7,4))
plt.hist(EI_N[~np.isnan(EI_N)], bins=40, edgecolor="k")
plt.xlabel("EI_N (m)"); plt.ylabel("freq")
plt.title("Distribuição do efeito indireto em N")
plt.grid(alpha=0.3)
plt.show()

plt.figure(figsize=(6,5))
sc = plt.scatter(lonP_deg, latP_deg, c=EI_N, s=10)
plt.colorbar(sc, label="EI_N (m)")
plt.xlabel("lon"); plt.ylabel("lat")
plt.title("EI_N nos pontos de cálculo")
plt.show()


In [ ]:
# 1) grade alvo (a partir dos pontos P que você tem)
latP_deg_grid = np.unique(latP_deg)   # latitudes únicas (graus)
lonP_deg_grid = np.unique(lonP_deg)   # longitudes únicas (graus)

# garantir ordem crescente
latP_deg_grid = np.sort(latP_deg_grid)
lonP_deg_grid = np.sort(lonP_deg_grid)

LATg, LONg = np.meshgrid(latP_deg_grid, lonP_deg_grid, indexing="ij")  # (nlat, nlon)

# 2) pontos de entrada (dispersos) e valores
pts = np.column_stack([latP_deg, lonP_deg])
vals = EI_N

# 3) interpolação para a grade (linear + fallback nearest para buracos)
EI_grid_lin = griddata(pts, vals, (LATg, LONg), method="linear")
EI_grid_nei = griddata(pts, vals, (LATg, LONg), method="nearest")
EI_grid = np.where(np.isnan(EI_grid_lin), EI_grid_nei, EI_grid_lin)

# 4) empacotar em DataArray usando os NOMES corretos
da_EI = xr.DataArray(
    EI_grid,
    dims=("y", "x"),
    coords={"y": latP_deg_grid, "x": lonP_deg_grid},
    name="EI_N_m",
    attrs={"units": "m", "description": "Efeito indireto em N (interpolado p/ grade do modelo)"},
)

da_EI


In [ ]:
plt.figure(figsize=(10,6))
da_EI.plot(cmap="PuOr", cbar_kwargs={"label":"EI_N (m)"})
plt.title("EI_N (m) na grade do modelo")
plt.show()

In [ ]:
md.dataarray_to_geotiff(da_EI, "../data/processed/EI_N.tif")
print("[OK] ../data/processed/EI_N.tif")
